# 利用灰階影像製作數字特徵字典

In [1]:
from tqdm import tqdm_notebook as tqdm
from itertools import chain
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import glob
import json
import cv2
import os

In [2]:
def create_Pattern_Dict():
        image_PD = cv2.imread("Data/001742/001742 083012 p.png", 0)
        image_CAL = cv2.imread("Data/026550/026550 033117 p.png", 0)
            
        w, h = 15, 12

        location_PD = [
            (651, 606), (634, 798), (634, 734), (634, 754), (634, 778), 
            (634, 200), (634, 176), (535, 842), (182, 92), (83, 175)
        ]
        
        location_CAL = [
            (286, 90), (754, 798), (754, 880), (754, 842), (754, 476), 
            (754, 606), (754, 412), (754, 946), (302, 732), (590, 925)
        ]
        
        def crop_image_2_dict(image, locations):
                pattern_dict = {}
                for idx, location in enumerate(locations):
                        x, y = location
                        crop_image  = image [ x : x+h , y : y+w  ]
                        pattern  = get_pattern_by_black_pixel(crop_image)
                        pattern_dict[pattern] = idx
                return pattern_dict
            
        def get_pattern_by_black_pixel(image):
                bi_image = np.where(image == 240 , 255, 0)
                unique, counts = np.unique(bi_image, return_counts=True)
                freq = np.asarray((unique, counts)).T
                return int(freq[0][1])
        
        PD_dict   = crop_image_2_dict(image_PD , location_PD )
        CAL_dict = crop_image_2_dict(image_CAL, location_CAL)
        return PD_dict, CAL_dict                

In [3]:
PD_dict, CAL_dict = create_Pattern_Dict()

In [4]:
PD_dict

{44: 0, 29: 1, 33: 4, 35: 3, 36: 5, 38: 6, 25: 7, 42: 8, 37: 9}

In [5]:
CAL_dict

{44: 0, 21: 1, 31: 2, 33: 4, 36: 5, 38: 6, 25: 7, 42: 8, 37: 9}

### 由此可知，2, 3, 4 最容易搞混，pixel 數相近，另外是 10 以上未辨識

# 找出每個表格的表格位置並且存在字典裡面

In [6]:
def find_table_start_point(index_of_240):
        start_x, start_y = 0, 0

        i = 0
        max_threshold, min_threshold = 80, 40
        total_len = len(index_of_240[0])
        while i < total_len:
                sequence_len = 1
                while i + sequence_len < total_len:
                        if index_of_240[1][i] + sequence_len != index_of_240[1][i+sequence_len]:
                            break
                        sequence_len += 1

                if  max_threshold >= sequence_len >= min_threshold:
                        start_x = index_of_240[0][i]
                        # start_y = index_of_240[1][i]
                        break

                i += sequence_len

        index = index_of_240[0].tolist().index(start_x)
        while True:
                if  index_of_240[1][index] + 1 == index_of_240[1][ index + 1 ] and index_of_240[1][index] + 2 == index_of_240[1][ index + 2 ] and \
                        index_of_240[1][index] + 3 == index_of_240[1][ index + 3 ] and index_of_240[1][index] + 4 == index_of_240[1][ index + 4 ] and \
                        index_of_240[1][index] + 5 == index_of_240[1][ index + 5 ] and index_of_240[1][index] + 6 == index_of_240[1][ index + 6 ]:

                        start_y = index_of_240[1][index]
                        break
                index += 1

        return (start_x, start_y)

def get_subseq_pair(index_of_240, start_value, direction):
        start_end_pair = []
        all_index = None
        if direction == 'x':
                all_index = [ j for i, j in zip(index_of_240[0], index_of_240[1]) if i == start_value ]
        if direction == 'y':
                all_index = [ i for i, j in zip(index_of_240[0], index_of_240[1]) if j == start_value ]

        start = end = 0 # start index and end index
        total_len = len(all_index)
        i = 0
        while i < total_len:
                if ( (i+1) < total_len and (all_index[i] + 1) != all_index[i+1]) or (i+1 == total_len):
                        end = i
                        distance = all_index[end] - all_index[start]
                        if distance > 7:
                            start_end_pair.append((all_index[start], all_index[end]))
                        start = end = i+1
                i+=1

        return start_end_pair

def points_to_cell(point_row_list, point_col_list):
        row = 0
        x = [ int(i) for i in point_row_list ]
        y = [ int(i) for i in point_col_list ]

        total_row = len(point_row_list)
        table_cells = []
        while row < total_row:
                col = 0
                while col + 1 < 32:
                        position = {
                            "location": (col//2, row//2),
                            "corner": [
                                (x[row]  , y[col]  ), # upper-left  
                                (x[row]  , y[col+1]), # upper-right
                                (x[row+1], y[col]),   # lower-left
                                (x[row+1], y[col+1])  # lower-right
                            ]
                        }
                        table_cells.append(position)
                        col += 2
                row += 2
        return table_cells
    
def get_table_cell_location(image_path):
        
        image = cv2.imread(image_path, 0)
        index_of_240 = np.where(image==240)
        start_point = find_table_start_point(index_of_240)


        point_column     = get_subseq_pair(index_of_240, start_point[0], 'x') # left -> right
        point_row        = get_subseq_pair(index_of_240, start_point[1], 'y') # top -> down    
        point_column_second = get_subseq_pair(index_of_240, point_row[1][0], 'x')

        point_column_list = [ i for i in list(chain(*point_column)) if i >= start_point[1] ]
        point_row_list = [ i for i in list(chain(*point_row)) if i >= start_point[0] ]
        point_column_second_list = [ i for i in list(chain(*point_column_second)) if i >= start_point[1] ]
        table_cells = points_to_cell(point_row_list, point_column_second_list)

        image = np.where(image==240, 255, 0)
        image_color = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2RGB)
        for cell in table_cells:
            for point in cell["corner"]:
                 image_color[point] = [0, 0, 255]
        
        return table_cells

In [7]:
def statistic_cell_size(image_path ,table_situation, cell_dict):
        
        heights, widths = [], []
        for item in cell_dict:
                width = item["corner"][1][1] - item["corner"][0][1]
                widths.append(width)

                height = item["corner"][2][0] - item["corner"][0][0]
                heights.append(height)

        width_freq = np.array(np.unique(widths, return_counts=True)).T
        height_freq = np.array(np.unique(heights, return_counts=True)).T
        cell_same_size = True if len(height_freq) == 1 else False

        item_info = {
            "dir_name": image_path[:image_path.rfind("\\")],
            "cell_width" : width_freq[0, 0],
            "cell_height": height_freq[0, 0],
            "cell_same_size": cell_same_size
        }

        table_situation = table_situation.append(item_info, ignore_index=True)
        return table_situation
    


table_situation = pd.DataFrame(columns=["dir_name", "cell_width", "cell_height", "cell_same_size"])
files = [ i for i in glob.iglob("Data/*/*.png") ]
for image_path in tqdm(files):
        table_cell = get_table_cell_location(image_path)
        table_situation = statistic_cell_size(image_path, table_situation, table_cell)
        
        
        if len(table_cell) != 512:
                print("%s don't have correct number of table cell" % files)
                raise ValueError
              

In [8]:
valid_condition = (table_situation.cell_width > 50) &  (table_situation.cell_width < 70) & (table_situation.cell_height > 10) & (table_situation.cell_height < 20)
valid = table_situation[valid_condition] 
valid

,dir_name,cell_width,cell_height,cell_same_size
1,Data\000411,58,11,False
2,Data\001742,61,13,True
5,Data\004499,61,11,False
6,Data\005627,61,13,True
7,Data\007274,59,13,True
...,...,...,...,...
294,Data\S469585_1,61,13,True
295,Data\S469585_2,61,13,True
296,Data\S594966_1,61,13,True
297,Data\S594966_2,61,13,True


目前只有 289 筆可以用

# Mapping Pattern to Number

In [9]:
def image_to_data(image_path, table_info, cal_dict, pd_dict):
    
        image = cv2.imread(image_path, 0)
        image = np.where(image == 240, 255, 0)
        
        tooth_info = { i:{
                            "lingual":{
                                        "CAL": 0,
                                        "PD" : 0
                                     },
                            "buccal" :{
                                        "CAL": 0,
                                        "PD" : 0
                                     }
                        } 
                        for i in range(1, 33, 1)
                    }


        lower_count = 31
        cal_upper_loc = check_swap_cal(image, table_info, cal_dict, "upper") # to do
        cal_lower_loc = check_swap_cal(image, table_info, cal_dict, "lower")


        for cell in table_info:
                if cell["location"][1] not in [3, 5, 10, cal_upper_loc, cal_lower_loc, 21, 26, 28]:
                    continue

                origin = cell["corner"][0]
                width = cell["corner"][1][1] - cell["corner"][0][1]

                block_1 = image[origin[0]: origin[0]+14, origin[1]          : origin[1]+ width//3    ]
                block_2 = image[origin[0]: origin[0]+14, origin[1]+ width//3: origin[1]+ width//3 * 2]
                block_3 = image[origin[0]: origin[0]+14, origin[1]+ width//3 * 2: origin[1]+ width   ]

                first, second, third = 0, 0, 0

                if cell["location"][1] in [3, cal_upper_loc, cal_lower_loc, 28]:
                        first  = search_pattern_dic(block_1, cal_dict, "CAL")
                        second = search_pattern_dic(block_2, cal_dict, "CAL")
                        third  = search_pattern_dic(block_3, cal_dict, "CAL")

                else:
                        first  = search_pattern_dic(block_1, pd_dict,  "PD")
                        second = search_pattern_dic(block_2, pd_dict,  "PD")
                        third  = search_pattern_dic(block_3, pd_dict,  "PD")


                if first is None or second is None or third is None:
                        print("Value Cannot Recognization in ")

                if cell["location"][1] == 3: # upper, back, cal
                        tooth_num = cell["location"][0] + 1
                        tooth_info[tooth_num]["buccal"]["CAL"] = [first, second, third]

                if cell["location"][1] == 5: # upper, back, pd
                        tooth_num = cell["location"][0] + 1
                        tooth_info[tooth_num]["buccal"]["PD"] = [first, second, third]

                if cell["location"][1] == 10: # upper, front, pd
                        tooth_num = cell["location"][0] + 1
                        tooth_info[tooth_num]["lingual"]["PD"] = [first, second, third]

                if cell["location"][1] == cal_upper_loc: # upper, front, cal
                        tooth_num = cell["location"][0] + 1
                        tooth_info[tooth_num]["lingual"]["CAL"] = [first, second, third]

                if cell["location"][1] == cal_lower_loc: # lower, front, cal
                        tooth_num = cell["location"][0] + 1 + lower_count
                        tooth_info[tooth_num]["lingual"]["CAL"] = [first, second, third]
                        lower_count = (lower_count - 2) % 32

                if cell["location"][1] == 21: # lower, front, pd
                        tooth_num = cell["location"][0] + 1 + lower_count
                        tooth_info[tooth_num]["lingual"]["PD"] = [first, second, third]
                        lower_count = (lower_count - 2) % 32

                if cell["location"][1] == 26: # lower, back, pd
                        tooth_num = cell["location"][0] + 1 + lower_count
                        tooth_info[tooth_num]["buccal"]["PD"] = [first, second, third]
                        lower_count = (lower_count - 2) % 32

                if cell["location"][1] == 28: # lower, back, cal
                        tooth_num = cell["location"][0] + 1 + lower_count
                        tooth_info[tooth_num]["buccal"]["CAL"] = [first, second, third]
                        lower_count = (lower_count - 2) % 32
    
        return tooth_info
    
    
def search_pattern_dic(array, pattern_dict, mode):
        white_matrix = np.full(array.shape, 255)
        if np.array_equal(array, white_matrix):
                return -99

        else:
                unique, counts = np.unique(array, return_counts=True)
                
                freq = np.asarray((unique, counts)).T
                black_count = int(freq[0][1])
                
                if int(freq[0][1]) > 50:
                        loc = np.where(array == 0)
                        y = np.count_nonzero(loc[0] == loc[0][-1])
                        if y == 2:
                                return 10
                        if y == 6:
                                return 11
                        if y == 9:
                                return 12
                        return 13

                elif black_count in pattern_dict.keys():
                        if pattern_dict[black_count] not in [2, 3, 4]:
                                return pattern_dict[black_count]

                        else: 
                                loc = np.where(array == 0)
                                y = np.count_nonzero(loc[0] == loc[0][-1])
                                if y == 2:
                                        return 4
                                if y == 6:
                                        return 2
                                if y == 5:
                                        return 3

        return -9999

def check_swap_cal(image, table_info, pattern_dict, direction):
    
        loc = 13 if direction == "upper" else 18

        count = 0
        for cell in table_info:
                if cell["location"][1] == loc:
                        origin = cell["corner"][0]
                        width = cell["corner"][1][1] - cell["corner"][0][1]

                        block_1 = image[origin[0]: origin[0]+14, origin[1]          : origin[1]+ width//3    ]
                        block_2 = image[origin[0]: origin[0]+14, origin[1]+ width//3: origin[1]+ width//3 * 2]
                        block_3 = image[origin[0]: origin[0]+14, origin[1]+ width//3 * 2: origin[1]+ width   ]

                        first  = search_pattern_dic(block_1, pattern_dict, "CAL")
                        second = search_pattern_dic(block_2, pattern_dict, "CAL")
                        third  = search_pattern_dic(block_3, pattern_dict, "CAL")

                        if first == -99: 
                                count += 1
                        if second == -99: 
                                count += 1
                        if third == -99: 
                                count += 1

                if count > 30:
                        if direction == "upper":
                                loc -= 1
                        else: 
                                loc += 1
                        break

        return loc

In [10]:
def tell_eight_zero_ambiguous(image_path, table_dict):
        eight_detection = False
        for num, situation in table_dict.items():
                if situation["lingual"]["CAL"].count(8) > 1:
                        situation["lingual"]["CAL"] = [ 0 if x==8 else x for x in situation["lingual"]["CAL"] ]
                        eight_detection = True
                        
                if situation["buccal"]["CAL"].count(8) > 1:
                        situation["buccal"]["CAL"] = [ 0 if x==8 else x for x in situation["lingual"]["CAL"] ]
                        eight_detection = True
        
        if eight_detection:
                print(image_path)
        return table_dict

In [11]:
class facade_table_recognition:
        def __init__(self):
                self.table_situation = pd.DataFrame(columns=["dir_name", "cell_width", "cell_height", "cell_same_size"])
                self.valid_list = None
        
        def run(self):
                PD_dict, CAL_dict = create_Pattern_Dict()
                files = [ i for i in glob.iglob("Data/*/*.png") ]
                for image_path in tqdm(files, desc="Table Cell Locating Task"):
                        table_cell = get_table_cell_location(image_path)
                        self.table_situation = statistic_cell_size(image_path, self.table_situation, table_cell)
                        assert len(table_cell) == 512
                
                self.valid_list = self.table_situation[(self.table_situation.cell_width  > 50) & (self.table_situation.cell_width  < 70) & (self.table_situation.cell_height > 10) & (self.table_situation.cell_height < 20)] 
                valid_tables = [  table_png for i in list(self.valid_list["dir_name"]) for table_png in glob.iglob("%s/*.png" % i) ]
                for image_path in tqdm(valid_tables, desc="Table Recognition Task"):
                        table_cell = get_table_cell_location(image_path)
                        recognition_result = image_to_data(image_path, table_cell, CAL_dict, PD_dict)
                        recognition_result = tell_eight_zero_ambiguous(image_path, recognition_result)
                        save_path = "%s\\table_info.json" % (image_path[:image_path.rfind("\\")])
                        with open(save_path, 'w', encoding='utf-8') as f:
                                json.dump(recognition_result, f, indent=4)

In [12]:
facade = facade_table_recognition()
facade.run()

Data\001742\001742 083012 p.png
Data\007274\007274 030116 p.png
Data\007501\007501 100716 p.png
Data\008908\008908 042709 p.png
Data\010801\010801 020119 p.png
Data\0109363\0109363 102318 p.png
Data\010953\010953 041218 p.png
Data\014185\014185 100417 p.png
Data\016165\016165 091516 p.png
Data\017070\017070 010616 p.png
Data\017374\017374 020717 p.png
Data\017635\017635 112112 p.png
Data\019462\019462 031209 p.png
Data\019660\019660 042319 p.png
Data\021033\021033 070915 p.png
Data\021271\021271 101613 p.png
Data\022198_2\022198_2 091819 p.png
Data\022702\022702 091218 p.png
Data\025179_1\025179_1 092010 p.png
Data\025575\025575 082514 p.png
Data\026110_1\026110_1 042213 p.png
Data\026110_2\026110_2 050619 p.png
Data\026127\026127 012214 p.png
Data\033168\033168 062818 p.png
Data\035015_1\035015_1 030413 p.png
Data\035015_2\035015_2 100319 p.png
Data\036214\036214 022713 p.png
Data\036574\036574 021914 p.png
Data\037535\037535 041417 p.png
Data\043521_2\043521_2 081418 p.png
Data\04519